# TPCH Datas

## Database Entities, Relationships, and Characteristics

- TPCH_SF1: Consists of the base row size (several million elements).
- TPCH_SF10: Consists of the base row size x 10.
- TPCH_SF100: Consists of the base row size x 100 (several hundred million elements).
- TPCH_SF1000: Consists of the base row size x 1000 (several billion elements).

<img src="https://docs.snowflake.com/en/_images/sample-data-tpch-schema.png" alt="Schema for TPC-H benchmark data" style="zoom:50%;" />

In [1]:
%pip install pyarrow duckdb

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 134.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [3]:
import duckdb
import pyarrow.parquet as pq
con = duckdb.connect(database=':memory:')
con.execute("INSTALL tpch; LOAD tpch")
con.execute("CALL dbgen(sf=1)")

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 10, Finished, Available, Finished)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
con.execute("show tables").fetchall()

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 12, Finished, Available, Finished)

[('customer',),
 ('lineitem',),
 ('nation',),
 ('orders',),
 ('part',),
 ('partsupp',),
 ('region',),
 ('supplier',)]

In [8]:
tables = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
for t in tables:
    #res = con.query("SELECT * FROM " + t)
    con.sql("COPY (SELECT * FROM " + t + ") TO '/lakehouse/default/Files/TPCH/" + t + ".parquet' ")

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 17, Finished, Available, Finished)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 18, Finished, Available, Finished)

In [9]:
df = spark.read.parquet("Files/TPCH/orders.parquet")
display(df)

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, be4ff1d4-75f9-42a5-9b7e-c25a8e267cb1)

In [10]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 20, Finished, Available, Finished)

In [11]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_orders'

df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Quarter', quarter(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))

df.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 21, Finished, Available, Finished)

In [13]:
%%sql
SELECT count(*) FROM LoadTests.fact_orders

StatementMeta(, 71d5db72-6465-4c88-8013-72ce12240c91, 23, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>